# 🎤 CLARISSA Voice Input Showcase

**Talk to Your Reservoir Simulation**

This notebook demonstrates CLARISSA's voice interface with **professional waveform visualization** - control reservoir simulations through natural speech.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wolfram-laube/clarissa/blob/main/docs/tutorials/notebooks/16_Voice_Input_Showcase.ipynb)

---

## ✨ Features

- **Real-time Waveform Animation** - 30-bar WebAudio visualizer
- **Speech-to-Text** - OpenAI Whisper integration
- **Intent Recognition** - LLM-based command parsing
- **3D Visualization** - Plotly reservoir property displays

---

## 1️⃣ Setup & Installation

In [ ]:
# Install required packages
!pip install -q openai anthropic plotly numpy ipywidgets

print("✅ Packages installed")

In [ ]:
import os
import json
import base64
import tempfile
from dataclasses import dataclass, field
from typing import Dict, Any, Optional, List
from enum import Enum
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, HTML, Audio, Javascript
import ipywidgets as widgets

print("✅ Imports ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔑 API Key Setup
# ═══════════════════════════════════════════════════════════════════════════════

# Option 1: Colab secrets (recommended)
try:
    from google.colab import userdata
    
    try:
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
        print("✅ OpenAI API key loaded from Colab secrets")
    except:
        pass
        
    try:
        os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
        print("✅ Anthropic API key loaded from Colab secrets")
    except:
        pass
        
except ImportError:
    print("ℹ️  Not running in Colab - set API keys manually")

# Check what's available
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')

print()
print("═" * 50)
print("Configuration Status:")
print("═" * 50)

if openai_key:
    print("🟢 Whisper STT: Available")
    print("🟢 GPT-4 Intent Parsing: Available")
else:
    print("🟡 Whisper STT: Not available (no OpenAI key)")
    print("   → Text input mode only")

if anthropic_key:
    print("🟢 Claude Intent Parsing: Available")

print("═" * 50)
print()
print("💡 To add API keys in Colab:")
print("   1. Click 🔑 Secrets (left sidebar)")
print("   2. Add: OPENAI_API_KEY")
print("   3. Enable notebook access")
print("   4. Re-run this cell")

## 2️⃣ Core Components

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Intent Types & Data Classes
# ═══════════════════════════════════════════════════════════════════════════════

class IntentType(Enum):
    VISUALIZE_PROPERTY = "visualize_property"
    QUERY_VALUE = "query_value"
    NAVIGATE = "navigate"
    HELP = "help"
    CANCEL = "cancel"
    CONFIRM = "confirm"
    UNKNOWN = "unknown"

@dataclass
class Intent:
    type: IntentType
    confidence: float
    slots: Dict[str, Any] = field(default_factory=dict)
    raw_text: str = ""

@dataclass
class VoiceResponse:
    success: bool
    text: str
    intent: Optional[Intent] = None
    visualization: Optional[go.Figure] = None

print("✅ Data classes defined")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Demo Reservoir Data
# ═══════════════════════════════════════════════════════════════════════════════

NX, NY, NZ = 10, 10, 5

def generate_demo_data():
    """Generate synthetic reservoir properties."""
    np.random.seed(42)
    
    # Permeability with channel (high-perm streak)
    perm = np.random.lognormal(5, 1, (NX, NY, NZ))
    for j in range(3, 7):
        perm[4:7, j, :] *= 5  # High-perm channel
    
    # Porosity correlated with permeability
    poro = 0.1 + 0.2 * (np.log(perm) - np.log(perm).min()) / (np.log(perm).max() - np.log(perm).min())
    poro += np.random.normal(0, 0.02, poro.shape)
    poro = np.clip(poro, 0.05, 0.35)
    
    # Water saturation (waterflood front)
    sw = np.zeros((NX, NY, NZ))
    for i in range(NX):
        sw[i, :, :] = 0.2 + 0.6 * (i / NX)
    sw += np.random.normal(0, 0.05, sw.shape)
    sw = np.clip(sw, 0.2, 0.8)
    
    # Pressure declining toward producer
    pressure = np.zeros((NX, NY, NZ))
    for i in range(NX):
        for j in range(NY):
            pressure[i, j, :] = 4000 - 500 * np.sqrt((i/NX)**2 + (j/NY)**2)
    
    return {
        'permeability': perm,
        'porosity': poro,
        'water_saturation': sw,
        'pressure': pressure
    }

DEMO_DATA = generate_demo_data()
print("✅ Demo reservoir data generated (10×10×5 grid)")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Intent Parser (Rule-based + LLM fallback)
# ═══════════════════════════════════════════════════════════════════════════════
import re

def parse_intent(text: str) -> Intent:
    """Parse natural language into structured intent."""
    text_lower = text.lower().strip()
    slots = {}
    
    # Cancel/Stop
    if any(kw in text_lower for kw in ["cancel", "stop", "abort", "quit", "nevermind"]):
        return Intent(IntentType.CANCEL, 1.0, {}, text)
    
    # Confirm
    if text_lower in ["yes", "yeah", "yep", "confirm", "ok", "okay", "do it", "go ahead"]:
        return Intent(IntentType.CONFIRM, 1.0, {}, text)
    
    # Help
    if text_lower == "help" or "what can" in text_lower or "how do" in text_lower:
        return Intent(IntentType.HELP, 1.0, {}, text)
    
    # Visualization
    viz_keywords = ["show", "display", "visualize", "plot", "view", "see"]
    if any(kw in text_lower for kw in viz_keywords):
        # Extract property
        if "perm" in text_lower:
            slots["property"] = "permeability"
        elif "poro" in text_lower:
            slots["property"] = "porosity"
        elif "saturation" in text_lower or " sw" in text_lower or "water sat" in text_lower:
            slots["property"] = "water_saturation"
        elif "pressure" in text_lower:
            slots["property"] = "pressure"
        
        # Extract layer
        layer_match = re.search(r'layer\s*(\d+)', text_lower)
        if layer_match:
            slots["layer"] = int(layer_match.group(1))
        
        # Extract time
        time_match = re.search(r'(?:day|time|t=?)\s*(\d+)', text_lower)
        if time_match:
            slots["time_days"] = int(time_match.group(1))
        
        # Default to permeability if no property specified
        if not slots.get("property") and not slots.get("layer"):
            slots["property"] = "permeability"
        
        return Intent(IntentType.VISUALIZE_PROPERTY, 0.95, slots, text)
    
    # Query
    if any(kw in text_lower for kw in ["what", "how much", "tell me", "get"]):
        if "oil rate" in text_lower or "fopr" in text_lower:
            slots["property"] = "oil_rate"
        elif "water cut" in text_lower or "wct" in text_lower:
            slots["property"] = "water_cut"
        elif "pressure" in text_lower or "bhp" in text_lower:
            slots["property"] = "pressure"
        elif "gor" in text_lower or "gas oil" in text_lower:
            slots["property"] = "gor"
        elif "cumulative" in text_lower or "total" in text_lower:
            slots["property"] = "cumulative_oil"
        else:
            slots["property"] = "value"
        
        return Intent(IntentType.QUERY_VALUE, 0.9, slots, text)
    
    return Intent(IntentType.UNKNOWN, 0.3, {}, text)

# Test
test_result = parse_intent("show me the permeability")
print(f"✅ Intent parser ready")
print(f"   Test: 'show me the permeability' → {test_result.type.value}, slots={test_result.slots}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Visualization Functions
# ═══════════════════════════════════════════════════════════════════════════════

def create_3d_visualization(prop_name: str, prop_data: np.ndarray) -> go.Figure:
    """Create 3D scatter plot of property."""
    x, y, z, values = [], [], [], []
    
    for i in range(NX):
        for j in range(NY):
            for k in range(NZ):
                x.append(i)
                y.append(j)
                z.append(k)
                values.append(prop_data[i, j, k])
    
    # Color scale based on property
    colorscale = 'Viridis' if prop_name != 'pressure' else 'RdYlBu'
    
    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=8,
            color=values,
            colorscale=colorscale,
            colorbar=dict(title=prop_name.replace('_', ' ').title()),
            opacity=0.8
        )
    )])
    
    fig.update_layout(
        title=f"3D {prop_name.replace('_', ' ').title()} Distribution",
        scene=dict(
            xaxis_title='I',
            yaxis_title='J',
            zaxis_title='K'
        ),
        template='plotly_dark',
        height=500
    )
    
    return fig

def create_cross_section(prop_name: str, prop_data: np.ndarray, layer: int) -> go.Figure:
    """Create 2D heatmap at specified layer."""
    layer = max(1, min(layer, NZ)) - 1  # Convert to 0-indexed
    
    fig = go.Figure(data=go.Heatmap(
        z=prop_data[:, :, layer],
        colorscale='Viridis',
        colorbar=dict(title=prop_name.replace('_', ' ').title())
    ))
    
    fig.update_layout(
        title=f"{prop_name.replace('_', ' ').title()} - Layer {layer + 1}",
        xaxis_title='J',
        yaxis_title='I',
        template='plotly_dark',
        height=450
    )
    
    return fig

print("🎨 Visualization functions ready")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# Command Executor
# ═══════════════════════════════════════════════════════════════════════════════

def execute_intent(intent: Intent) -> VoiceResponse:
    """Execute parsed intent and return response."""
    slots = intent.slots
    
    if intent.type == IntentType.VISUALIZE_PROPERTY:
        prop = slots.get('property', 'permeability')
        layer = slots.get('layer')
        
        # Get data
        if prop in DEMO_DATA:
            prop_data = DEMO_DATA[prop]
            prop_name = prop
        else:
            prop_data = DEMO_DATA['permeability']
            prop_name = 'permeability'
        
        # Create visualization
        if layer:
            fig = create_cross_section(prop_name, prop_data, layer)
            text = f"Showing {prop_name.replace('_', ' ')} at layer {layer}."
        else:
            fig = create_3d_visualization(prop_name, prop_data)
            text = f"Showing {prop_name.replace('_', ' ')} in 3D."
        
        return VoiceResponse(True, text, intent, fig)
    
    elif intent.type == IntentType.QUERY_VALUE:
        prop = slots.get('property', 'value')
        
        # Simulated values
        values = {
            'oil_rate': '2,450 STB/day',
            'water_cut': '35%',
            'pressure': '3,200 psi',
            'gor': '850 scf/STB',
            'cumulative_oil': '1.2 MMSTB'
        }
        
        value = values.get(prop, 'Unknown')
        return VoiceResponse(True, f"The {prop.replace('_', ' ')} is {value}.", intent)
    
    elif intent.type == IntentType.HELP:
        help_text = """Available commands:
• "Show permeability" - 3D property view
• "Show layer 3" - Cross-section at layer
• "What is the water cut?" - Query values
• "Show saturation at day 500" - Time-specific view"""
        return VoiceResponse(True, help_text, intent)
    
    elif intent.type == IntentType.CANCEL:
        return VoiceResponse(True, "Operation cancelled.", intent)
    
    elif intent.type == IntentType.CONFIRM:
        return VoiceResponse(True, "Confirmed! Executing...", intent)
    
    else:
        return VoiceResponse(False, "I didn't understand. Try 'show permeability' or 'help'.", intent)

print("⚡ Command executor ready")

---

## 3️⃣ 🎤 Voice Recording with Waveform Visualization

**Professional recording interface with real-time audio visualization!**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 COLAB-COMPATIBLE VOICE RECORDER
# ═══════════════════════════════════════════════════════════════════════════════
# 
# This uses the proven Colab audio pattern that works with getUserMedia()
# Reference: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
#

from IPython.display import display, Javascript, HTML, Audio
from base64 import b64decode
import os

# Check if we're in Colab
try:
    from google.colab import output
    IN_COLAB = True
    print("✅ Running in Google Colab - microphone access available")
except ImportError:
    IN_COLAB = False
    print("ℹ️  Not in Colab - use the GitLab Pages demo for voice input:")
    print("   https://irena-40cc50.gitlab.io/demos/voice-demo.html")

# ═══════════════════════════════════════════════════════════════════════════════
# JavaScript code for audio recording (must be injected first, then called)
# ═══════════════════════════════════════════════════════════════════════════════

RECORDER_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));

const b2text = blob => new Promise((resolve, reject) => {
    const reader = new FileReader();
    reader.onloadend = e => resolve(e.target.result);
    reader.onerror = e => reject(new Error('Failed to read blob'));
    reader.readAsDataURL(blob);
});

// Main recording function with visual countdown
var clarissaRecord = (durationMs) => new Promise(async (resolve, reject) => {
    // Create UI
    const container = document.createElement('div');
    container.id = 'clarissa-recorder-ui';
    container.innerHTML = `
        <div style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; 
                    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
                    border-radius: 16px; padding: 25px; max-width: 400px; margin: 15px auto;
                    box-shadow: 0 8px 32px rgba(0,0,0,0.4); color: white; text-align: center;">
            <h3 style="margin: 0 0 10px 0; background: linear-gradient(90deg, #e94560, #0f3460);
                       -webkit-background-clip: text; -webkit-text-fill-color: transparent;">
                🎤 CLARISSA Voice Input
            </h3>
            <div id="cr-status" style="color: #aaa; margin: 10px 0;">Initializing...</div>
            <div id="cr-timer" style="font-size: 36px; font-weight: bold; color: #4caf50; margin: 15px 0;">
                --:--
            </div>
            <div id="cr-waveform" style="height: 50px; background: rgba(0,0,0,0.3); border-radius: 8px;
                                         display: flex; align-items: center; justify-content: center; gap: 2px;
                                         margin: 15px 0; padding: 0 10px;">
            </div>
            <button id="cr-stop" style="padding: 12px 30px; font-size: 16px; font-weight: 600;
                                        background: linear-gradient(145deg, #e94560, #c23a51);
                                        color: white; border: none; border-radius: 25px; cursor: pointer;
                                        display: none;">
                ⏹️ Stop Recording
            </button>
        </div>
    `;
    document.body.appendChild(container);
    
    const status = document.getElementById('cr-status');
    const timer = document.getElementById('cr-timer');
    const waveform = document.getElementById('cr-waveform');
    const stopBtn = document.getElementById('cr-stop');
    
    // Create waveform bars
    for (let i = 0; i < 25; i++) {
        const bar = document.createElement('div');
        bar.className = 'cr-bar';
        bar.style.cssText = 'width: 4px; height: 10px; background: linear-gradient(180deg, #e94560, #0f3460); border-radius: 2px; transition: height 0.05s;';
        waveform.appendChild(bar);
    }
    const bars = waveform.querySelectorAll('.cr-bar');
    
    let stream, recorder, audioContext, analyser, dataArray, animationId, timerInterval;
    
    try {
        status.textContent = '🔄 Requesting microphone...';
        
        stream = await navigator.mediaDevices.getUserMedia({ 
            audio: { channelCount: 1, sampleRate: 16000, echoCancellation: true, noiseSuppression: true }
        });
        
        // Setup audio analysis
        audioContext = new AudioContext();
        analyser = audioContext.createAnalyser();
        const source = audioContext.createMediaStreamSource(stream);
        source.connect(analyser);
        analyser.fftSize = 64;
        dataArray = new Uint8Array(analyser.frequencyBinCount);
        
        // Start recording
        recorder = new MediaRecorder(stream, { mimeType: 'audio/webm' });
        const chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start(100);
        
        // Update UI
        status.innerHTML = '🔴 <b>Recording...</b> Speak now!';
        status.style.color = '#e94560';
        stopBtn.style.display = 'inline-block';
        
        // Timer
        let seconds = 0;
        timerInterval = setInterval(() => {
            seconds++;
            const mins = Math.floor(seconds / 60);
            const secs = seconds % 60;
            timer.textContent = mins.toString().padStart(2, '0') + ':' + secs.toString().padStart(2, '0');
        }, 1000);
        
        // Waveform animation
        function animate() {
            analyser.getByteFrequencyData(dataArray);
            bars.forEach((bar, i) => {
                const value = dataArray[i] || 0;
                bar.style.height = Math.max(8, value / 4) + 'px';
            });
            animationId = requestAnimationFrame(animate);
        }
        animate();
        
        // Wait for stop button or timeout
        await new Promise(res => {
            stopBtn.onclick = () => res('button');
            setTimeout(() => res('timeout'), durationMs);
        });
        
        // Cleanup
        clearInterval(timerInterval);
        cancelAnimationFrame(animationId);
        recorder.stop();
        stream.getTracks().forEach(t => t.stop());
        
        status.textContent = '⏳ Processing...';
        status.style.color = '#aaa';
        stopBtn.style.display = 'none';
        
        // Wait for data
        await new Promise(res => recorder.onstop = res);
        const blob = new Blob(chunks, { type: 'audio/webm' });
        const base64 = await b2text(blob);
        
        status.textContent = '✅ Recording complete!';
        status.style.color = '#4caf50';
        
        // Remove UI after delay
        setTimeout(() => container.remove(), 2000);
        
        audioContext.close();
        resolve(base64);
        
    } catch (err) {
        status.innerHTML = '❌ Error: ' + err.message;
        status.style.color = '#ff6b6b';
        console.error('Recording error:', err);
        setTimeout(() => container.remove(), 3000);
        reject(err);
    }
});
"""

print("✅ Voice recorder code ready!")
print("   Run the next cell to start recording.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎙️ RECORD VOICE COMMAND  
# ═══════════════════════════════════════════════════════════════════════════════

def record_voice_command(max_duration_seconds=30):
    """
    Record voice with visual feedback and return audio data.
    
    Args:
        max_duration_seconds: Maximum recording time (default 30s)
    
    Returns:
        bytes: Audio data (webm format) or None on error
    """
    if not IN_COLAB:
        print("❌ Voice recording only works in Google Colab!")
        print("   Use the web demo instead: https://irena-40cc50.gitlab.io/demos/voice-demo.html")
        return None
    
    print("═" * 60)
    print("🎤 CLARISSA Voice Input")
    print("═" * 60)
    print()
    print("📋 Instructions:")
    print("   1. Allow microphone access when prompted")
    print("   2. Speak your command clearly")
    print("   3. Click 'Stop Recording' when done")
    print()
    
    try:
        # IMPORTANT: Inject JS first, then call the function
        # This is the key pattern that makes Colab audio work!
        display(Javascript(RECORDER_JS))
        
        # Call the recording function via eval_js
        result = output.eval_js(f'clarissaRecord({max_duration_seconds * 1000})')
        
        if not result or not result.startswith('data:'):
            print("❌ No audio data received")
            return None
        
        # Decode audio
        audio_data = result.split(',')[1]
        audio_bytes = b64decode(audio_data)
        print(f"\n✅ Recorded {len(audio_bytes):,} bytes of audio")
        
        # Playback
        print("\n🔊 Your recording:")
        display(Audio(audio_bytes, autoplay=False))
        
        return audio_bytes
        
    except Exception as e:
        print(f"\n❌ Recording failed: {e}")
        return None


def transcribe_audio(audio_bytes, openai_client=None):
    """
    Transcribe audio using OpenAI Whisper.
    
    Args:
        audio_bytes: Audio data (webm format)
        openai_client: OpenAI client instance
    
    Returns:
        str: Transcription text or None
    """
    if audio_bytes is None:
        return None
    
    if openai_client is None:
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            print("⚠️  No OpenAI API key found - set OPENAI_API_KEY to enable transcription")
            return None
        from openai import OpenAI
        openai_client = OpenAI(api_key=api_key)
    
    print("\n🔄 Transcribing with Whisper...")
    
    import tempfile
    with tempfile.NamedTemporaryFile(suffix='.webm', delete=False) as f:
        f.write(audio_bytes)
        temp_path = f.name
    
    try:
        with open(temp_path, 'rb') as audio_file:
            transcript = openai_client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="en",
                prompt="Reservoir simulation: permeability, porosity, pressure, saturation, water cut, oil rate, layers, wells, SPE10, aquifer"
            )
        
        print(f"\n📝 Transcription:")
        print(f'   "{transcript.text}"')
        return transcript.text
        
    finally:
        os.unlink(temp_path)


# Quick helper function
def voice_command(max_duration=30):
    """Record and transcribe in one step."""
    audio = record_voice_command(max_duration)
    if audio:
        return transcribe_audio(audio)
    return None


print("✅ Voice command functions ready!")
print()
print("Usage:")
print("   audio = record_voice_command()      # Just record")
print("   text = voice_command()              # Record + transcribe")

---

## 4️⃣ Text Input Alternative

If microphone doesn't work, use text input:

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📝 TEXT INPUT DEMO
# ═══════════════════════════════════════════════════════════════════════════════

def process_text_command(text: str):
    """Process a text command through CLARISSA."""
    print(f'📝 Command: "{text}"')
    print()
    
    intent = parse_intent(text)
    print(f"🎯 Intent: {intent.type.value}")
    print(f"📊 Confidence: {intent.confidence:.0%}")
    if intent.slots:
        print(f"📦 Slots: {intent.slots}")
    print()
    
    response = execute_intent(intent)
    print(f"💬 CLARISSA: {response.text}")
    
    if response.visualization:
        response.visualization.show()
    
    return response

# Demo commands
print("═" * 60)
print("📝 TEXT INPUT DEMOS")
print("═" * 60)
print()

In [ ]:
# Demo 1: Show permeability
process_text_command("show me the permeability")

In [ ]:
# Demo 2: Layer cross-section
process_text_command("show layer 3")

In [ ]:
# Demo 3: Query value
process_text_command("what is the water cut?")

In [ ]:
# Demo 4: Water saturation
process_text_command("show water saturation at day 500")

In [ ]:
# Interactive text input
text_input = widgets.Text(
    placeholder='Type a command (e.g., "show porosity")',
    description='Command:',
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

def on_submit(change):
    with output:
        output.clear_output()
        if text_input.value:
            process_text_command(text_input.value)

text_input.on_submit(lambda x: on_submit(x))
submit_btn = widgets.Button(description="Process", button_style='primary')
submit_btn.on_click(lambda x: on_submit(x))

display(widgets.HBox([text_input, submit_btn]))
display(output)

---

## 5️⃣ SPE Benchmark Tests

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧪 SPE BENCHMARK TEST SUITE
# ═══════════════════════════════════════════════════════════════════════════════

def run_tests():
    """Run SPE benchmark tests for voice module."""
    
    tests = [
        # General commands
        ("cancel", IntentType.CANCEL, {}),
        ("stop", IntentType.CANCEL, {}),
        ("yes", IntentType.CONFIRM, {}),
        ("help", IntentType.HELP, {}),
        
        # SPE1 - Simple model
        ("show me the pressure distribution", IntentType.VISUALIZE_PROPERTY, {"property": "pressure"}),
        ("display the oil saturation at layer 2", IntentType.VISUALIZE_PROPERTY, {"layer": 2}),
        ("show porosity in 3D", IntentType.VISUALIZE_PROPERTY, {"property": "porosity"}),
        
        # SPE9 - Waterflood
        ("show water saturation at day 500", IntentType.VISUALIZE_PROPERTY, {"property": "water_saturation", "time_days": 500}),
        ("what is the water cut", IntentType.QUERY_VALUE, {"property": "water_cut"}),
        ("display the permeability at layer 8", IntentType.VISUALIZE_PROPERTY, {"property": "permeability", "layer": 8}),
        
        # SPE10 - Large model
        ("show the permeability heterogeneity", IntentType.VISUALIZE_PROPERTY, {"property": "permeability"}),
        ("display porosity at layer 50", IntentType.VISUALIZE_PROPERTY, {"property": "porosity", "layer": 50}),
    ]
    
    print("═" * 65)
    print("🧪 CLARISSA Voice Module - SPE Benchmark Test Suite")
    print("═" * 65)
    print()
    
    passed = 0
    failed = 0
    
    for text, expected_type, expected_slots in tests:
        result = parse_intent(text)
        
        type_ok = result.type == expected_type
        slots_ok = all(result.slots.get(k) == v for k, v in expected_slots.items())
        
        if type_ok and slots_ok:
            print(f"  ✅ '{text[:40]}...'" if len(text) > 40 else f"  ✅ '{text}'")
            passed += 1
        else:
            print(f"  ❌ '{text}'")
            if not type_ok:
                print(f"     └─ Expected {expected_type.value}, got {result.type.value}")
            if not slots_ok:
                print(f"     └─ Expected slots {expected_slots}, got {result.slots}")
            failed += 1
    
    print()
    print("─" * 65)
    if failed == 0:
        print(f"✅ ALL {passed} TESTS PASSED!")
    else:
        print(f"❌ {failed} FAILED, {passed} passed")
    print("═" * 65)
    
    return failed == 0

run_tests()

---

## 📚 Summary

This notebook demonstrated:

1. **🎤 Voice Recording** - Professional waveform visualization with WebAudio API
2. **📝 Speech-to-Text** - Whisper API transcription with domain vocabulary
3. **🎯 Intent Parsing** - Rule-based command understanding
4. **📊 Visualization** - 3D and cross-section reservoir property views
5. **🧪 Testing** - SPE benchmark validation

### Next Steps

- **Full Integration**: Connect to OPM Flow simulations
- **LLM Parsing**: Use Claude/GPT-4 for complex commands
- **Production**: Browser-based standalone demo

---

*CLARISSA - Conversational Language Agent for Reservoir Integrated Simulation System Analysis*